In [9]:
import numpy as np

n=3


A1 = np.linspace(0,10,n)
A2 = np.linspace(0,10,n)
A3 = np.linspace(0,10,n)
A4 = np.linspace(0,10,n)
A5 = np.linspace(0,10,n)
A6 = np.linspace(0,10,n)

#print(A1)





mat = np.zeros((n,n,n))
print(mat)

for j in range(n):
    for i in range(n):
        mat[j,:,i] += A1
        mat[j,i,:] += A1


print(mat)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
[[[ 0.  5. 10.]
  [ 5. 10. 15.]
  [10. 15. 20.]]

 [[ 0.  5. 10.]
  [ 5. 10. 15.]
  [10. 15. 20.]]

 [[ 0.  5. 10.]
  [ 5. 10. 15.]
  [10. 15. 20.]]]


In [6]:
import numpy as np

n=700


A1 = np.linspace(0,10,n)
A2 = np.linspace(0,10,n)
A3 = np.linspace(0,10,n)
A4 = np.linspace(0,10,n)
A5 = np.linspace(0,10,n)
A6 = np.linspace(0,10,n)

A1_grid, A2_grid, A3_grid= np.meshgrid(A1,A2,A3,indexing='ij')

A_grid = A1_grid + A2_grid + A3_grid
#print(A1_grid)
#print(A2_grid)
#print(A3_grid)
print(A_grid)

[[[0.00000000e+00 1.43061516e-02 2.86123033e-02 ... 9.97138770e+00
   9.98569385e+00 1.00000000e+01]
  [1.43061516e-02 2.86123033e-02 4.29184549e-02 ... 9.98569385e+00
   1.00000000e+01 1.00143062e+01]
  [2.86123033e-02 4.29184549e-02 5.72246066e-02 ... 1.00000000e+01
   1.00143062e+01 1.00286123e+01]
  ...
  [9.97138770e+00 9.98569385e+00 1.00000000e+01 ... 1.99427754e+01
   1.99570815e+01 1.99713877e+01]
  [9.98569385e+00 1.00000000e+01 1.00143062e+01 ... 1.99570815e+01
   1.99713877e+01 1.99856938e+01]
  [1.00000000e+01 1.00143062e+01 1.00286123e+01 ... 1.99713877e+01
   1.99856938e+01 2.00000000e+01]]

 [[1.43061516e-02 2.86123033e-02 4.29184549e-02 ... 9.98569385e+00
   1.00000000e+01 1.00143062e+01]
  [2.86123033e-02 4.29184549e-02 5.72246066e-02 ... 1.00000000e+01
   1.00143062e+01 1.00286123e+01]
  [4.29184549e-02 5.72246066e-02 7.15307582e-02 ... 1.00143062e+01
   1.00286123e+01 1.00429185e+01]
  ...
  [9.98569385e+00 1.00000000e+01 1.00143062e+01 ... 1.99570815e+01
   1.99713